# Lab 9

Name: Muhammad Bazly Bin Burhan
Student ID: SW01081224

## Example 1

### 1. Import Required Libraries

In [26]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# For topic modeling
from gensim import corpora
from gensim.models import LdaModel

import pandas as pd

# Download the NLTK resources
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bazly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bazly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bazly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 2. Load the Data

In [27]:
# Load the data
documents = [
 "Rafael Nadal Joins Roger Federer in Missing U.S. Open",
 "Rafael Nadal Is Out of the Australian Open",
 "Biden Announces Virus Measures",
 "Biden's Virus Plans Meet Reality",
 "Where Biden's Virus Plan Stands"
]

### 3. Do Data Preprocessing

In [28]:
# Do data preprocessing on the documents
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower()) # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()] # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words] # Filter out stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Lemmatize the tokens
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]
preprocessed_documents
   

[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

### 4. Crate a Document-Term Matrix

In [29]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

### 5. Run LDA

In [30]:
# corpus: bag of words representation of the documents
# num_topics: number of topics to be extracted by the model
# id2word=dictionary: dictionary mapping form word IDs to words
# passes: number of passes through the corpus during training

# Train the LDA model on the corpus with 2 topics using Gensim's LdaModel class
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

### 6. Interpret the Results

In [31]:
# Empty list to store dominant topic labels for each document
article_labels = []

#  Iterate over the preprocessed documents
for i, doc in enumerate(preprocessed_documents):
    # Fore each document, convert to box representation
    bow = dictionary.doc2bow(doc)
    # Get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # Determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # Append to the list
    article_labels.append(dominant_topic)

In [32]:
# Create DataFrame
df = pd.DataFrame({'Document': documents, 'Topic': article_labels})

# Display the DataFrame
print("Table with Articles and Topic:")
print(df)

Table with Articles and Topic:
                                            Document  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      0
1         Rafael Nadal Is Out of the Australian Open      0
2                     Biden Announces Virus Measures      1
3                   Biden's Virus Plans Meet Reality      1
4                    Where Biden's Virus Plan Stands      1


In [33]:
# Print the top terms for each topic
print("Top terms for each topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top terms for each topic:
Topic 0:
- "rafael" (weight: 0.131)
- "nadal" (weight: 0.131)
- "open" (weight: 0.131)
- "federer" (weight: 0.079)
- "roger" (weight: 0.079)
- "join" (weight: 0.079)
- "missing" (weight: 0.079)
- "australian" (weight: 0.078)
- "virus" (weight: 0.027)
- "biden" (weight: 0.027)

Topic 1:
- "biden" (weight: 0.166)
- "virus" (weight: 0.166)
- "plan" (weight: 0.119)
- "stand" (weight: 0.071)
- "measure" (weight: 0.071)
- "announces" (weight: 0.071)
- "meet" (weight: 0.071)
- "reality" (weight: 0.071)
- "australian" (weight: 0.025)
- "open" (weight: 0.024)



## Example 2

### 1. Import Required Libraries

In [34]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# For topic modeling
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd
# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bazly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bazly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bazly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 2. Load the Data

In [35]:
df = pd.read_csv('npr.csv')
documents = df['Article'].tolist()

### 3. Do Data Preprocessing

In [36]:
stop_words = set(stopwords.words('english')) # Create a set of English stopwords
lemmatizer = WordNetLemmatizer() # Initialize a WordNet lemmatizer

def preprocess_text(text):
    tokens = word_tokenize(text.lower()) # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()] # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words] # Remove stopwords from the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Lemmatize each token
    return tokens # Return the preprocessed tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents] # Preprocess each document in the list
print(preprocessed_documents[0])

['washington', '2016', 'even', 'policy', 'bipartisan', 'politics', 'sense', 'year', 'show', 'little', 'sign', 'ending', 'president', 'obama', 'moved', 'sanction', 'russia', 'alleged', 'interference', 'election', 'concluded', 'republican', 'long', 'called', 'similar', 'severe', 'measure', 'could', 'scarcely', 'bring', 'approve', 'house', 'speaker', 'paul', 'ryan', 'called', 'obama', 'measure', 'appropriate', 'also', 'overdue', 'prime', 'example', 'administration', 'ineffective', 'foreign', 'policy', 'left', 'america', 'weaker', 'eye', 'gop', 'leader', 'sounded', 'much', 'theme', 'urging', 'president', 'obama', 'year', 'take', 'strong', 'action', 'deter', 'russia', 'worldwide', 'aggression', 'including', 'operation', 'wrote', 'devin', 'nunes', 'chairman', 'house', 'intelligence', 'committee', 'week', 'left', 'office', 'president', 'suddenly', 'decided', 'stronger', 'measure', 'indeed', 'appearing', 'cnn', 'frequent', 'obama', 'critic', 'trent', 'frank', 'called', 'much', 'tougher', 'acti

### 4. Create Document-Term Matrix

In [37]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents) 

# Filter out tokens that appear in less than 15 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=15, no_above=0.5)

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents] 

### 5. Run LDA

In [38]:
# Run LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15) # Train an LDA model on the corpus with 2 topics using Gensim's LdaModel class

### 6. Interpret Results

In [39]:
# empty list to store dominant topic labels for each document
article_labels = []

# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # for each document, convert to bag-of-words representation
    bow = dictionary.doc2bow(doc)
    # get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # append to the list
    article_labels.append(dominant_topic)

# Create DataFrame
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()

Table with Articles and Topic:
                                                 Article  Topic
0      In the Washington of 2016, even when the polic...      2
1        Donald Trump has used Twitter  —   his prefe...      2
2        Donald Trump is unabashedly praising Russian...      2
3      Updated at 2:50 p. m. ET, Russian President Vl...      2
4      From photography, illustration and video, to d...      1
...                                                  ...    ...
11987  The number of law enforcement officers shot an...      4
11988    Trump is busy these days with victory tours,...      2
11989  It’s always interesting for the Goats and Soda...      0
11990  The election of Donald Trump was a surprise to...      2
11991  Voters in the English city of Sunderland did s...      2

[11992 rows x 2 columns]



In [40]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()


Top terms for Topic #0:
['country', 'world', 'human', 'study', 'disease', 'war', 'scientist', 'two', 'group', 'report']

Top terms for Topic #1:
['know', 'think', 'thing', 'life', 'really', 'woman', 'story', 'day', 'back', 'show']

Top terms for Topic #2:
['trump', 'clinton', 'president', 'republican', 'state', 'campaign', 'election', 'obama', 'vote', 'voter']

Top terms for Topic #3:
['health', 'percent', 'school', 'company', 'state', 'care', 'student', 'program', 'child', 'million']

Top terms for Topic #4:
['police', 'law', 'court', 'state', 'case', 'report', 'officer', 'right', 'told', 'justice']



In [41]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()


Top Terms for Each Topic:
Topic 0:
- "country" (weight: 0.004)
- "world" (weight: 0.004)
- "human" (weight: 0.004)
- "study" (weight: 0.003)
- "disease" (weight: 0.003)
- "war" (weight: 0.003)
- "scientist" (weight: 0.003)
- "two" (weight: 0.003)
- "group" (weight: 0.003)
- "report" (weight: 0.003)

Topic 1:
- "know" (weight: 0.005)
- "think" (weight: 0.005)
- "thing" (weight: 0.005)
- "life" (weight: 0.004)
- "really" (weight: 0.004)
- "woman" (weight: 0.004)
- "story" (weight: 0.004)
- "day" (weight: 0.003)
- "back" (weight: 0.003)
- "show" (weight: 0.003)

Topic 2:
- "trump" (weight: 0.030)
- "clinton" (weight: 0.012)
- "president" (weight: 0.011)
- "republican" (weight: 0.008)
- "state" (weight: 0.008)
- "campaign" (weight: 0.008)
- "election" (weight: 0.006)
- "obama" (weight: 0.006)
- "vote" (weight: 0.005)
- "voter" (weight: 0.005)

Topic 3:
- "health" (weight: 0.008)
- "percent" (weight: 0.007)
- "school" (weight: 0.007)
- "company" (weight: 0.007)
- "state" (weight: 0.005)
- "